In [1]:
import os
import sys
import re
import time
from pymongo import MongoClient
from github import Github
from random import randint
from pprint import pprint

# Credentials

In [2]:
pw_file = 'credentials/mongo_pw.txt'
if os.path.exists(pw_file): 
    with open(pw_file, 'r') as f:
        pub_ip, mongo_usr, mongo_usr_pw = f.readline().strip().split(', ')

# Connect to MongoDB

In [3]:
# connect to ec2 mongo client
client = MongoClient('{0}:27017'.format(pub_ip))

# get reference to  resume_db
db = client['github_db']

# authenticate user for database
db.authenticate(mongo_usr, mongo_usr_pw)

True

# Create a MongoDB collection

In [4]:
def initialize_collections(db, reset=False):
    # drop collections if reset is set to True
    if reset:
        for name in db.collection_names():
            db.drop_collection(name)
            
    # create a collection in the 'github_db' database (not required but helpful)
    lst_collections = ['git_users_meta', 'git_users_following', 'git_users_followers', 
                       'git_repos_meta', 'git_repos_docs', 'git_repos_subscribers', 'git_repos_contributors']

    # initialize blank collections
    for collection in lst_collections:
        if collection not in db.collection_names():
            db.create_collection(collection)

In [5]:
initialize_collections(db)

In [6]:
def show_collections():
    '''
    IN: None
    RETURN: Printed output of collections and collection counts
    '''
    for name in sorted(db.collection_names()):
        print('Documents in \"{0}\" collection: {1}'.format(name, db[name].count()))

In [7]:
show_collections()

Documents in "git_repos_contributors" collection: 3
Documents in "git_repos_docs" collection: 88
Documents in "git_repos_meta" collection: 3
Documents in "git_repos_subscribers" collection: 3
Documents in "git_users_followers" collection: 1
Documents in "git_users_following" collection: 1
Documents in "git_users_meta" collection: 1


# Get authenticated access to Github for higher requests limit

In [8]:
# import github private token
with open('credentials/token.txt', 'r') as infile:
    token = infile.readline().strip()
    user = infile.readline().strip()

git_client = Github(token)

# Make MongoDB database copy

In [9]:
def copy_db(current_db, new_db):
    r = client.admin.command('copydb', fromdb=current_db, todb=new_db)
    if r['ok'] == 1.0:
        return print('DB {0} created'.format(new_db))

# Rate limits

In [10]:
def get_remaining(_type ='core'):
    '''
    IN: term to determine if rate limit for requests (core) or searching (search) is to be returned
        _type => 'core' or 'search'
    RETURN: tuple of remaining rate limit quantity, and time till reset
    '''
    rate_limit = git_client.get_rate_limit()
    raw = dict(rate_limit.raw_data)
    remaining = int(raw['resources'][_type]['remaining'])
    
    return remaining

In [11]:
def show_rate_remaining():
    '''
    IN: None
    RETURN: Printed output of github api rate limits
    '''
    print('Resource rate limit remaining: {0}'.format(get_remaining('core')))
    print('Search rate limit remaining: {0}'.format(get_remaining('search')))

In [12]:
show_rate_remaining()

Resource rate limit remaining: 5000
Search rate limit remaining: 30


# Wait function

In [13]:
def limit_wait(location, ct=0):
    '''
    IN: 
        location => function name (input text) where program paused till rate is reset
        ct => counter to show length of pause in 5 minute intervals
    RETURN: 
        None - program will halt here if limit is below buffer threshold and continue while it is above/reset
    '''
    # wait till rate limit is refreshed
    rate = get_remaining('core')
    
    if rate < 150: #buffer
        print('Stopped @ {0}'.format(location))
    while (rate < 150):
        print('Rate: {0} ===> Waiting...{1}'.format(rate, ct))
        time.sleep(300)
        rate = get_remaining('core')
        ct += 5

# Repo Metadata

In [14]:
def repo_metadata(repo):
    '''
    IN: github repository object
    RETURN: dictionary with only relevant/desired repo metadata to be uploaded into mongodb
    '''
    repo_keys = ['owner', 'name', 'full_name', 'description', 'fork', 'html_url', 'homepage', 'language', 
                 'forks_count', 'size', 'open_issues_count', 'has_issues', 'has_wiki', 'has_downloads', 
                 'pushed_at', 'created_at', 'subscribers_count', 'stargazers_count']
    
    owner_keys = ['type', 'login', 'id', 'site_admin']
    
    # check for rate limit
    limit_wait('repo_metadata')
    
    # get returned payload raw data into dictionary
    raw_dict = dict(repo.raw_data)
    
    # trim repo by keys listed above
    repo_dict = {k: raw_dict[k] for k in repo_keys}
    
    # trim owner by keys listed above
    owner_dict = {k: raw_dict['owner'][k] for k in owner_keys}
    repo_dict['owner'] = owner_dict
    repo_dict['_id'] = repo.id
    
    return repo_dict

In [15]:
def repo_metadata_upsert(repo):
    '''
    IN: github repository object
    RETURN: None - data returned from repo_metadata function is inserted into mongodb
    '''
    col = db['git_repos_meta']
    try:
        col.update_one(
            {'_id': repo.id}, 
            {'$set':{col.insert_one(repo_metadata(repo))}}, 
            upsert=True)
    except:
        pass
    return None

# Repo Subscribers

In [16]:
def repo_subscribers(repo):
    '''
    IN: github repository object
    RETURN: dictionary containing repo subscribers to be uploaded into mongodb
    '''
    # check for rate limit
    limit_wait('repo_subscribers')
    
    repo_dict = {}
    repo_dict['_id'] = repo.id
    repo_dict['repo_fullname'] = repo.full_name
    repo_dict['repo_name'] = repo.name
    repo_dict['repo_owner_login'] = repo.owner.login
    
    try:
        repo_dict['repo_subscribers'] = [f.login for f in repo.get_subscribers()]
        
    except:
        repo_dict['repo_subscribers'] = []
        
    return repo_dict

In [17]:
def repo_subscribers_upsert(repo):
    '''
    IN: github repository object
    RETURN: None - data returned from repo_subscribers function is inserted into mongodb
    '''
    col = db['git_repos_subscribers']
    try:
        col.update_one(
            {'_id': repo.id}, 
            {'$set': {col.insert_one(repo_subscribers(repo))}
            }, upsert=True)
    except:
        pass
    return None

# Repo Contributors

In [18]:
def repo_contributors(repo):
    '''
    IN: github repository object
    RETURN: dictionary containing contributors (login names) keyed by repo full_name to be uploaed into mongodb
    '''
    # check for rate limit
    limit_wait('repo_contributors')
    
    repo_dict = {}
    repo_dict['_id'] = repo.id
    repo_dict['repo_fullname'] = repo.full_name
    repo_dict['repo_name'] = repo.name
    repo_dict['repo_owner_login'] = repo.owner.login
    
    try:
        repo_dict['repo_contributors'] = [f.login for f in repo.get_contributors()]
        
    except:
        repo_dict['repo_contributors'] = ''
        
    return repo_dict

In [19]:
def repo_contributors_upsert(repo):
    '''
    IN: github repository object
    RETURN: None - data returned from repo_contributors function is inserted into mongodb
    '''
    col = db['git_repos_contributors']
    try:
        col.update_one(
            {'_id': repo.id}, 
            {'$set': {col.insert_one(repo_contributors(repo))}
            }, upsert=True)
    except:
        pass
    return None

# User Metadata

In [20]:
def user_metadata(user):
    '''
    IN: github NamedUser object
    RETURN: dictionary with only relevant/desired user metadata to be uploaded into mongodb
    '''
    user_keys = ['email', 'followers', 'hireable', 'login', 'bio', 'avatar_url', 'company', 
                 'updated_at', 'type', 'created_at', 'name', 'location', 'html_url', 'public_repos', 
                 'blog', 'public_gists', 'following']
    
    # check for rate limit
    limit_wait('user_metadata')
        
    # get returned payload raw data into dictionary
    raw_dict = dict(user.raw_data)
    
    # trim user by keys listed above
    user_dict = {k: raw_dict[k] for k in user_keys}
    user_dict['_id'] = user.id
    
    return user_dict

In [21]:
def user_metadata_upsert(user):
    '''
    IN: github NamedUser object
    RETURN: None - data returned from user_metadata function is inserted into mongodb
    '''
    col = db['git_users_meta']
    try:
        col.update_one(
            {'_id': user.id}, 
            {'$set':{col.insert_one(user_metadata(user))}}, 
            upsert=True)
    except:
        pass
    return None

# User Following

In [22]:
def user_following(user):
    '''
    IN: github NamedUser object
    RETURN: dictionary containing users (login names) followed by user (passed) to be uploaed into mongodb
    '''
    # check for rate limit
    limit_wait('user_following') 
    
    user_dict = {}
    user_dict['_id'] = user.id
    user_dict['login'] = user.login
        
    try:
        user_dict['user_following'] = [f.login for f in repo.get_following()]
        
    except:
        user_dict['user_following'] = []
        
    return user_dict

In [23]:
def user_following_upsert(user):
    '''
    IN: github NamedUser object
    RETURN: None - data returned from user_following function is inserted into mongodb
    '''
    col = db['git_users_following']
    try:
        col.update_one(
            {'_id': user.id}, 
            {'$set': {col.insert_one(user_following(user))}
            }, upsert=True)
    except:
        pass
    return None

# User Followers

In [24]:
def user_followers(user):
    '''
    IN: github NamedUser object
    RETURN: dictionary containing users (login names) who follow user (passed) to be uploaed into mongodb
    '''
    # check for rate limit
    limit_wait('user_followers')
    
    user_dict = {}
    user_dict['_id'] = user.id
    user_dict['login'] = user.login
        
    try:
        user_dict['user_followers'] = [f.login for f in repo.get_followers()]
        
    except:
        user_dict['user_followers'] = []
    
    return user_dict

In [25]:
def user_followers_upsert(user):
    '''
    IN: github NamedUser object
    RETURN: None - data returned from user_followers function is inserted into mongodb
    '''
    col = db['git_users_followers']
    try:
        col.update_one(
            {'_id': user.id}, 
            {'$set': {col.insert_one(user_followers(user))}
            }, upsert=True)
    except:
        pass
    return None

# Repo Files

In [26]:
def repo_scripts_upsert(repo, file_name, doc_name, raw_file_cont, file_ext):
    '''
    IN: 
        repo ==> github repository object
        file_name ==> file name pulled from github repo
        raw_file_contnet ==> raw file contents of file_name
        file_ext ==> file extension of file_name
    RETURN: 
        None - file details/information uploaded into mongodb
    '''
    try:
        db['git_repos_docs'].update_one(
            {'doc_name' : doc_name}, 
            {'$set':{
                    'file_name': file_name,
                    'file_contents': raw_file_cont,
                    'file_extension': file_ext,
                    'repo_fullname' : repo.full_name,
                    'repo_id' : repo.id,
                    'repo_name' : repo.name,
                    'repo_owner_login' : repo.owner.login,
                    'doc_name' : doc_name}
                }, 
            upsert=True)
    except:
        pass
    return None

In [27]:
def repo_scripts(repo, _path='.'):
    '''
    IN: 
        repo ==> github repository object
        doc_ct ==> sequential number for 'clean' file names inserted into mongodb
        _path ==> path to folder to extract repository files; '.' is top level repository folder
    RETRUN: 
        None - all files from repo except those in the exclusion list will be inserted into mongob
    '''
    # only files with these extensions
    extensions = ['py', 'md', 'rst']
    
    # grab all contents in the main directory
    dir_contents = repo.get_dir_contents(_path)
    num_files = len(dir_contents)
    repo_fullname = repo.full_name

    # check for rate limit
    limit_wait('repo_scripts')
    
    for content in dir_contents:
        # if item is a directory then recursively navigate lower to get files inside
        if content.type == 'dir':
            repo_scripts(repo, _path=content.path)

        else:
            # get file extension
            file_ext = content.name.split('.')[-1]

            if file_ext in extensions:
                # try to decode, but return blank if fail
                try:
                    raw_file_content = content.decoded_content.decode(errors='replace')
                except:
                    raw_file_content = ''
                    
                # actual file name, mongodb doc stored name
                file_name = content.name
                doc_name = '{0}__{1}'.format(repo.id, file_name)
                
                # add to collection if under limit (>16mb)
                file_size = sys.getsizeof(raw_file_content)
                if file_size < 16000000:
                    repo_scripts_upsert(repo, file_name, doc_name, raw_file_content, file_ext)
                else:
                    print('FILE SIZE LIMIT: {0} -- {1}'.format(file_name, file_size))
    return None

# Search Github for 'Python' repositories

In [28]:
def get_repo(repo, ct, new_only=True):
    '''
    IN: 
        repo ==> github repository object
        new_only ==> True if only new repos are to be added, False if add all (new/existing)
    RETURN: None - all desired repository data uploaded to mongodb
    '''
    if new_only:
        if db['git_repos_meta'].find({'full_name': repo.full_name}).count() == 0:
            print('Repo (new).....#{0} - {1}'.format(ct, repo.full_name))
            repo_scripts(repo)
            repo_metadata_upsert(repo)
            repo_subscribers_upsert(repo)
            repo_contributors_upsert(repo)
    else:
        print('Repo (all).....#{0} - {1}'.format(ct, repo.full_name))
        repo_scripts(repo)
        repo_metadata_upsert(repo)
        repo_subscribers_upsert(repo)
        repo_contributors_upsert(repo)
        
    return None

In [29]:
def get_repo_owner(repo, ct, new_only=True):
    '''
    IN: github repository object
        repo ==> github repository object
        new_only ==> True if only new repos are to be added, False if add all (new/existing)
    RETURN: None - all desired data pertaining to repository owner uploaded to mongodb
    '''
    user = repo.owner
    if new_only:
        if db['git_users_meta'].find({'_id': user.id}).count() == 0:
            print('Repo owner (new).....#{0} - {1}'.format(ct, user.login))
            user_metadata_upsert(user)
            user_following_upsert(user)
            user_followers_upsert(user)
    else:
        print('Repo owner (all).....#{0} - {1}'.format(ct, user.login))
        user_metadata_upsert(user)
        user_following_upsert(user)
        user_followers_upsert(user)

In [37]:
def searched_repos(term='python', language='python', ct=0, new_only=True):
    '''
    IN: 
        term ==> term to search for within github
        language ==> limit to language identified (tagged by github)
        ct ==> counter initializer for number of repositories traversed
        new_only ==> only grab repositories not yet collected if True, otherwise grab all
    RETURN: None (print when finished traversing)
    '''
    for repo in git_client.search_repositories(term, sort='forks', order='desc'):
        try:
            if repo.language.lower() == language:
                ct+=1
                get_repo(repo, ct, new_only=new_only)
                get_repo_owner(repo, ct, new_only=new_only)
        except:
            pass
        
        # don't get cut off
        time.sleep(randint(2,9))
    return print('DONE')

In [39]:
searched_repos()

Repo (new).....#19 - kivy/kivy
Repo owner (new).....#19 - kivy
Repo (new).....#20 - quantopian/zipline
Repo (new).....#21 - pypa/pip
Repo (new).....#22 - echonest/pyechonest
Repo owner (new).....#22 - echonest
Repo (new).....#23 - andymccurdy/redis-py
Repo owner (new).....#23 - andymccurdy
Repo (new).....#24 - geekcomputers/Python
Repo owner (new).....#24 - geekcomputers
Repo (new).....#25 - omab/python-social-auth
Repo owner (new).....#25 - omab
Repo (new).....#26 - webpy/webpy
Repo owner (new).....#26 - webpy
Repo (new).....#27 - bokeh/bokeh
Repo owner (new).....#27 - bokeh
Repo (new).....#28 - spotify/luigi
Repo owner (new).....#28 - spotify
Repo (new).....#29 - joestump/python-oauth2
Repo owner (new).....#29 - joestump
Repo (new).....#30 - home-assistant/home-assistant
Repo owner (new).....#30 - home-assistant
Repo (new).....#31 - buildbot/buildbot
Stopped @ repo_scripts
Rate: 148 ===> Waiting...0
Rate: 148 ===> Waiting...5
Rate: 148 ===> Waiting...10
Rate: 148 ===> Waiting...15
Ra

KeyboardInterrupt: 

# Get repos under top level user/organization

In [ ]:
def repos_of_top_repos(ct=0, new_only=True):
    for user_meta in db['git_users_meta'].find():
        git_user = user_meta['login']
        _user = git_client.get_user(git_user)

        for repo in _user.get_repos():
            ct+=1
            get_repo(repo, ct, new_only=new_only)
            #get_repo_owner(repo) - not needed, already pulled from top_repos

            # don't get cut off
            time.sleep(randint(2,9))
    return print('DONE')

In [ ]:
repos_of_top_repos(new_only=True)